Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [16]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'not_mnist_dataset/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          /  predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.740246
Minibatch accuracy: 18.8%
Validation accuracy: 10.3%
Minibatch loss at step 50: 1.869945
Minibatch accuracy: 18.8%
Validation accuracy: 44.7%
Minibatch loss at step 100: 0.748052
Minibatch accuracy: 81.2%
Validation accuracy: 73.2%
Minibatch loss at step 150: 0.781204
Minibatch accuracy: 68.8%
Validation accuracy: 72.9%
Minibatch loss at step 200: 0.532637
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Minibatch loss at step 250: 0.480982
Minibatch accuracy: 87.5%
Validation accuracy: 77.1%
Minibatch loss at step 300: 1.524155
Minibatch accuracy: 56.2%
Validation accuracy: 78.7%
Minibatch loss at step 350: 1.007732
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 400: 0.935836
Minibatch accuracy: 75.0%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.371138
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.490868
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
M

### Best model accuracy 89.3%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    out_layer = tf.matmul(hidden, layer4_weights) + layer4_biases
    
    return out_layer
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.411534
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.311571
Minibatch accuracy: 12.5%
Validation accuracy: 24.4%
Minibatch loss at step 100: 1.362693
Minibatch accuracy: 56.2%
Validation accuracy: 61.0%
Minibatch loss at step 150: 0.999831
Minibatch accuracy: 62.5%
Validation accuracy: 67.3%
Minibatch loss at step 200: 0.720046
Minibatch accuracy: 87.5%
Validation accuracy: 71.3%
Minibatch loss at step 250: 0.576147
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 300: 1.263015
Minibatch accuracy: 62.5%
Validation accuracy: 75.6%
Minibatch loss at step 350: 1.298263
Minibatch accuracy: 62.5%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.709975
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.339050
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.392626
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Min

### Best model accuracy 89.2%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [14]:
batch_size = 128
patch_size = 5
depth = 16

num_hidden = 1024
num2_hidden = 256

beta_regul = 0.0005

initial_learning_rate = 0.1
decay_steps = 4000
decay_rate = 0.60

activation_funcs = {
    'relu': tf.nn.relu,
    'tanh': tf.nn.tanh,
}

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  keep_prob = tf.placeholder(tf.float32)
  is_training = tf.placeholder(tf.bool, name='is_training')
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

  # Variables.
  activation_func = activation_funcs['relu']
  regularizer = tf.contrib.layers.l2_regularizer(scale=beta_regul)

  weights = tf.get_variable("weights", [patch_size, patch_size, num_channels, depth], 
                            initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
  biases = tf.Variable(tf.zeros([depth]))

  weights2 = tf.get_variable("weights2", [patch_size, patch_size, depth, depth], 
                             initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
  biases2 = tf.Variable(tf.constant(1.0, shape=[depth]))

  weights3 = tf.get_variable("weights3", [image_size // 4 * image_size // 4 * depth, num_hidden], 
                             initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
  biases3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  weights4 = tf.get_variable("weights4", [num_hidden, num2_hidden], 
                             initializer=tf.contrib.layers.xavier_initializer(), regularizer = regularizer)
  biases4 = tf.Variable(tf.constant(1.0, shape=[num2_hidden]))

  weights5 = tf.get_variable("weights5", [num2_hidden, num_labels], 
                             initializer=tf.contrib.layers.xavier_initializer(), regularizer=regularizer)
  biases5 = tf.Variable(tf.constant(1.0, shape=[num_labels]))


  def conv(data, weights, stride):
        return tf.nn.conv2d(
            data,
            weights,
            stride,
            padding = 'SAME'
        )
    
  def dense(x, weights, biases):
    return tf.matmul(x, weights) + biases

  def fully_connected(x, weights, bias, activation):
        fc = dense(x, weights, bias)
        fc_act = activation(fc, 'activation')
        return tf.nn.dropout(fc_act, keep_prob)

  # Model.
  def model(data):
    conv1 = conv(data, weights, [1, 1, 1, 1])
    act1 = activation_func(conv1 + biases)
    pool1 = tf.nn.max_pool(act1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = conv(pool1, weights2, [1, 1, 1, 1])
    act2 = activation_func(conv2 + biases2)
    pool2 = tf.nn.max_pool(act2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    drop2 = tf.nn.dropout(pool2, keep_prob)
    
    #Fully connected
    shape = drop2.get_shape().as_list()
    reshape = tf.reshape(drop2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    fc = fully_connected(reshape, weights3, biases3, activation_func)
    fc2 = fully_connected(fc, weights4, biases4, activation_func)
    
    logits = dense(fc2, weights5, biases5)
    
    return logits

  # Training computation.
  logits = model(tf_train_dataset)
    
  reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
  reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)))
  loss+= reg_term
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [15]:
num_steps = 10001
batch_size = 128

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  feed_dict_test = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, is_training : False}
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5, is_training : True}
    #feed_dict_valid = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, is_training : False}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    #predictions_valid, = session.run([valid_prediction], feed_dict=feed_dict_valid)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      #print('Validation accuracy: %.1f%%' % accuracy(predictions_valid, valid_labels[:10000,:]))
    
  predictions_test, = session.run([test_prediction], feed_dict=feed_dict_test)
  print("")
  print('Test accuracy: %.1f%%' % accuracy(predictions_test, test_labels))

Initialized
Minibatch loss at step 0: 9.278179
Minibatch accuracy: 10.9%
Minibatch loss at step 50: 2.325947
Minibatch accuracy: 15.6%
Minibatch loss at step 100: 1.782419
Minibatch accuracy: 33.6%
Minibatch loss at step 150: 1.629246
Minibatch accuracy: 41.4%
Minibatch loss at step 200: 1.285987
Minibatch accuracy: 53.9%
Minibatch loss at step 250: 1.249285
Minibatch accuracy: 59.4%
Minibatch loss at step 300: 0.881200
Minibatch accuracy: 71.1%
Minibatch loss at step 350: 0.945781
Minibatch accuracy: 75.8%
Minibatch loss at step 400: 0.844932
Minibatch accuracy: 75.8%
Minibatch loss at step 450: 0.845954
Minibatch accuracy: 65.6%
Minibatch loss at step 500: 0.709268
Minibatch accuracy: 78.1%
Minibatch loss at step 550: 0.853300
Minibatch accuracy: 73.4%
Minibatch loss at step 600: 0.945700
Minibatch accuracy: 73.4%
Minibatch loss at step 650: 0.830394
Minibatch accuracy: 74.2%
Minibatch loss at step 700: 0.697428
Minibatch accuracy: 81.2%
Minibatch loss at step 750: 0.640641
Minibatch

Minibatch loss at step 6450: 0.382968
Minibatch accuracy: 86.7%
Minibatch loss at step 6500: 0.519897
Minibatch accuracy: 83.6%
Minibatch loss at step 6550: 0.389785
Minibatch accuracy: 89.1%
Minibatch loss at step 6600: 0.418968
Minibatch accuracy: 85.9%
Minibatch loss at step 6650: 0.266537
Minibatch accuracy: 91.4%
Minibatch loss at step 6700: 0.406275
Minibatch accuracy: 89.8%
Minibatch loss at step 6750: 0.279040
Minibatch accuracy: 90.6%
Minibatch loss at step 6800: 0.329159
Minibatch accuracy: 90.6%
Minibatch loss at step 6850: 0.409956
Minibatch accuracy: 88.3%
Minibatch loss at step 6900: 0.388263
Minibatch accuracy: 88.3%
Minibatch loss at step 6950: 0.362362
Minibatch accuracy: 89.8%
Minibatch loss at step 7000: 0.464256
Minibatch accuracy: 84.4%
Minibatch loss at step 7050: 0.534892
Minibatch accuracy: 85.2%
Minibatch loss at step 7100: 0.425715
Minibatch accuracy: 89.1%
Minibatch loss at step 7150: 0.329687
Minibatch accuracy: 89.8%
Minibatch loss at step 7200: 0.348910
Mi

### Best model accuracy 95.5%